In [1]:
import cv2
import pytesseract
import re
import numpy as np
import pandas as pd
import joblib
import easyocr

pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

In [2]:
def extract_img_1(path, keywords):

        reader = easyocr.Reader(['en'],gpu=False) # this needs to run only once to load the model into memory
        img = path
        output = reader.readtext(img,detail=1)
        dictionary = {}
        for i in output:
            probability = i[-1]
            
            if probability >= 0.6:
                text = i[1] 

            for keyword in keywords :
                try :
                    try :
                        if re.search(keyword, text, re.IGNORECASE):
                            if '=' in text :
                                list_  = text.split('=')
                                list_ = str(list_)
                                result = re.search('\d+\.{0,1}\d*', list_).group()
                                dictionary[keyword] = float(result)
                                break
                            elif ':' in text :
                                list_  = text.split(':')
                                list_ = str(list_)
                                result = re.search('\d+\.{0,1}\d*', list_).group()
                                dictionary[keyword] = float(result)
                                break             
                            elif output[output.index(i) + 1] [1] == '8' :
                                dictionary[keyword] = float(output[output.index(i) + 2] [1])  
                                break                                         
                    except : 
                        if re.search(keyword, text, re.IGNORECASE):
                            list_  = float(text.split())
                            result = re.search('\d+\.{0,1}\d*', list_).group()
                            dictionary[keyword] = float(result)
                            break                    
                except :
                    continue
        return dictionary   

In [4]:
def extract_img_2(path, keywords):
    img = cv2.imread(path)
    # Convert the image to gray scale
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    ocr_output = pytesseract.image_to_string(img_gray)
    dictionary = {}
    for line in ocr_output.split('\n'):
        for keyword in keywords:
            try :
                try :
                    if re.search(keyword, line, re.IGNORECASE):
                        if '=' in line :
                            list_  = line.split('=')
                            list_ = str(list_)
                            result = re.search('\d+\.{0,1}\d*', list_).group()
                            dictionary[keyword] = float(result)
                            break
                        elif ':' in line :
                            list_  = line.split(':')
                            list_ = str(list_)
                            result = re.search('\d+\.{0,1}\d*', list_).group()
                            dictionary[keyword] = float(result)
                            break          
                except : 
                    if re.search(keyword, line, re.IGNORECASE):
                        list_  = float(line.split())
                        result = re.search('\d+\.{0,1}\d*', list_).group()
                        dictionary[keyword] = float(result)
                        break                    
            except :
                continue
                
    return dictionary

In [5]:
extract_img_2('Sample\S2.jpg', ['Anti CPP'])

{'Anti CPP': 32.0}

In [6]:
pred_mapping = {'1' : 'Rheumatiod_Arthities'  , 
                '0' : 'Normal' }
try : 
        try :

                    keywords = ['Anti CPP']
                    extracted_dic = extract_img_1('Sample/S2.jpg', keywords)
                    model = joblib.load('Model/Rheumatiod.h5')
                    features  = list(model.feature_names_in_)
                    data_dic = {}
                    data_dic[features[0]] = extracted_dic['Anti CPP']
                    data_dic[features[1]] = np.random.choice([0,1])
                    custom_data = pd.DataFrame(data = [data_dic] )   
                    print('-----'*10)
                    print('Your Input :\n',custom_data)
                    print('-----'*10)       
                    output = model.predict(custom_data)[0] 
                    output = pred_mapping[str(output)]     
                    print('\t Output :',output)

        except :  
                    keywords =  ['Anti CPP']
                    extracted_dic = extract_img_2('Sample/S2.jpg',keywords)
                    model = joblib.load('Model/Rheumatiod.h5')
                    features  = list(model.feature_names_in_)
                    data_dic = {}
                    data_dic[features[0]] = extracted_dic['Anti CPP']
                    data_dic[features[1]] = np.random.choice([0,1])
                    custom_data = pd.DataFrame(data = [data_dic] )   
                    print('-----'*10)
                    print('Your Input :\n',custom_data)
                    print('-----'*10)       
                    output = model.predict(custom_data)[0]    
                    output = pred_mapping[str(output)]                            
                    print('\t Output :',output)
except : 
        print('Your uploaded image can\'t be detected')
        print('-----'*10)       
        model = joblib.load('Model/Rheumatiod.h5')
        features  = list(model.feature_names_in_)        
        print('\t Enter Manually the Following Please : ',features[:-1])


--------------------------------------------------
Your Input :
    Anti_CPP(u/ml)  Gender
0            32.0       1
--------------------------------------------------
	 Output : Rheumatiod_Arthities
